In [87]:
import matplotlib.pyplot as plt
from scipy import ndimage, misc
import numpy as np
import pandas as pd

In [195]:
originaldata = pd.read_csv('./Apparel/apparel-trainval.csv')
traindata = np.array(originaldata)[:48000,1:]
image = traindata[0]

#image_resized = misc.imresize(image, (32, 32))
#plt.imshow(image_resized)
#print(image_resized.shape)

In [197]:
image = image.reshape((28,28,1))
print(image.T.shape)

(1, 28, 28)


In [215]:
class CNN:
    def __init__(self,convLayerNum,filterChoice):
        self.convLayerNum = convLayerNum
        self.filterChoice = filterChoice
    
    def getGaussian(self,numberOfFilters,shape=(5,5),sigma=1):
        self.filterSize = shape[0]
        gaussianList = []
        for i in range(0,numberOfFilters):
            #########
            #python version of 'matlab - gaussian kernal generation'
            m,n = [(ss-1.)/2. for ss in shape]
            y,x = np.ogrid[-m:m+1,-n:n+1]
            h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
            h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
            sumh = h.sum()
            if sumh != 0:
                h /= sumh
            #########    
            #print("sigma = ",sigma)
            sigma = sigma - 0.05
            gaussianList.append(h)
        return gaussianList
    
    def randomFilter(self,numberOfFilters,depth):
        randomList = []
        for i in range(0,numberOfFilters):
            randomList.append(np.random.randn(5,5,depth))
        return randomList
    
    def convolution(self,imageInput,filters,stride):
        rowstride = 0
        colstride = 0
        listOfConvl = []
        convmat = []
        filterSize = 5
        #print("filterSize = ",len(filters))
        for filt in filters:
            rowstride=0
            convmat = []
            while rowstride+filterSize <= imageInput.shape[0]:
                colstride = 0
                while colstride+filterSize <= imageInput.shape[1]:
                    mat = imageInput[rowstride:rowstride+filterSize,\
                                     colstride:colstride+filterSize]
                    convmat.append(np.sum(np.multiply(mat,filt)))
                    colstride+=stride
                rowstride+=stride
            #print("lenth of filtered mat = ",len(convmat))
            convmat = np.array(convmat)
            filteredmatshape = imageInput.shape[0]-np.array(filt).shape[0]+1
            convmat = convmat.reshape((filteredmatshape,filteredmatshape))
            #print("convmat shape",convmat.shape)
            listOfConvl.append(convmat)
        return listOfConvl 
    
    def maxpool(self,outputs,stride):
        rowstride = 0
        colstride = 0
        listOfConvl = []
        convmat = []
        poolSize = 2
        #print("filterSize = ",len(filters))
        for output in outputs:
            rowstride=0
            convmat = []
            while rowstride+poolSize <= output.shape[0]:
                colstride = 0
                while colstride+poolSize <= output.shape[1]:
                    mat = output[rowstride:rowstride+poolSize,\
                                     colstride:colstride+poolSize]
                    convmat.append(np.max(mat))
                    colstride+=stride
                rowstride+=stride
            #print("lenth of filtered mat = ",len(convmat))
            convmat = np.array(convmat)
            filteredmatshape = int(np.sqrt(convmat.shape[0]))
            convmat = convmat.reshape((filteredmatshape,filteredmatshape))
            listOfConvl.append(convmat)
        return listOfConvl
    
    def feedforward(self,imageInput,numberOfFilters):
        #strides is an array of strides at every convl layer 
        convStride = 1
        poolStride = 2
        print("Input Layer",imageInput.shape)
        for numFilt in numberOfFilters:
            #Convolution
            #filters = self.getGaussian(numFilt)
            depth = imageInput.shape[2]
            filters = self.randomFilter(numFilt,depth)
            outputs = self.convolution(imageInput,filters,convStride)    
            #MaxPool
            maxoutputs = self.maxpool(outputs,poolStride)
            imageInput = np.array(maxoutputs).T
            print("Input Layer",imageInput.shape)
        #FeedforwardNN

In [ ]:
#inputs = image
#iterations = 2
#while i in range(0,iterations):
#    filters = getGaussian(size,numberOfFilters)
#    outputs = convolution(input,filters,stride)
#    maxoutputs = maxpool(outputs)

In [216]:
a = CNN(1,'gaussian')
a.feedforward(image,[6,16])

Input Layer (28, 28, 1)
Input Layer (12, 12, 6)
Input Layer (4, 4, 16)


In [ ]:
ndimage.filters.gaussian_filter()